In [2]:
import requests
import time
import json

# Replace with your OpenWeatherMap API key
api_key = '5d2fc15be838d558b62c2eda8ac3f326'


# Function to fetch weather data for a city
def fetch_weather(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}"
    response = requests.get(url)
    return response.json()

# Read city names from the file
with open('./get city data/top_900_cities_by_population.txt', 'r') as file:
    cities = [line.strip() for line in file.readlines()]

# Data storage
weather_data = []

# API call limits
calls_per_minute = 60
delay = 60 / calls_per_minute  # delay in seconds

# Fetch weather data for each city
for city in cities:
    try:
        data = fetch_weather(city, api_key)
        weather_data.append(data)
        time.sleep(delay)  # Respect the API rate limit
    except Exception as e:
        print(f"Error fetching data for {city}: {e}")

# Save the data to a JSON file
with open('weather_data.json', 'w') as outfile:
    json.dump(weather_data, outfile)

print("Data fetching complete.")


Data fetching complete.


In [1]:
import pandas as pd
import json

# Load your JSON data from the file
with open('weather_data.json', 'r') as file:
    data = json.load(file)

# Assuming 'data' is a list of records, where each record corresponds to a city's weather data
flattened_data = []

for record in data:
    # Flatten the 'list' key in your JSON (which contains the weather data)
    try:
        weather_data = pd.json_normalize(record,'list')

        del weather_data["weather"]

        weather_data["weather"] = pd.json_normalize(record['list'],"weather")["main"]
    
        
        # Flatten the 'city' data and join with weather data
        city_data = pd.json_normalize(record['city'])[["name","coord.lat","coord.lon"]]
        city_data = pd.concat([city_data]*len(weather_data), ignore_index=True)
        
        # print(pd.json_normalize(record['city']))

        # Combine city data with weather data
        combined_data = pd.concat([city_data, weather_data], axis=1)
        flattened_data.append(combined_data)

    except:
        print("Null record encountered. Proceeding anyways")

# Concatenate all the flattened data
df = pd.concat(flattened_data, ignore_index=True)

# Display the DataFrame
df.head()


Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways


,name,coord.lat,coord.lon,dt,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,...,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.3h,weather,snow.3h
0,New York,40.7143,-74.006,1701226800,10000,0.0,2023-11-29 03:00:00,273.13,267.10,273.13,...,50,-0.23,0,7.25,280,12.91,n,NaN,Clear,NaN
1,New York,40.7143,-74.006,1701237600,10000,0.0,2023-11-29 06:00:00,273.04,267.15,272.87,...,49,0.17,1,6.88,278,12.48,n,NaN,Clear,NaN
2,New York,40.7143,-74.006,1701248400,10000,0.0,2023-11-29 09:00:00,272.59,266.95,272.32,...,52,0.27,2,6.07,278,9.18,n,NaN,Clear,NaN
3,New York,40.7143,-74.006,1701259200,10000,0.0,2023-11-29 12:00:00,272.21,267.09,272.21,...,52,0.00,5,4.94,272,7.52,d,NaN,Clear,NaN
4,New York,40.7143,-74.006,1701270000,10000,0.0,2023-11-29 15:00:00,273.48,268.73,273.48,...,46,0.00,22,4.85,255,6.23,d,NaN,Clouds,NaN


In [2]:
df.to_csv("train_data.csv",index=False)